In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Masking,BatchNormalization, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import BertTokenizer
from keras.callbacks import EarlyStopping
import codecs
from keras.layers.convolutional import Conv2D
import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)


# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [3]:
# other parameters
#MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 768 #1024
nb_words = 3438
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# word_index = tokenizer.word_index
# nb_words = len(word_index) +1
# print('Found %s unique tokens' % len(word_index))

def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()

# # load model
# EMBEDDING_DIM = 300
# file_loc = '../../data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
# split = 7869
# earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
#                           restore_best_weights=True)

# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_text = x_train_text[mask]
# model: GRU


In [4]:
#split train/test
split = 8000

In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(2, 2), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(2, 2), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(2, 2), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(2, 2), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(2, 2), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [7]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('stmBERT_multi8000-3.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 251s 39ms/step - loss: 2.9989 - v_loss: 0.9991 - a_loss: 0.9998 - d_loss: 1.0000 - v_ccc: 9.3581e-04 - a_ccc: 1.8938e-04 - d_ccc: -2.4937e-06 - val_loss: 2.9765 - val_v_loss: 0.9934 - val_a_loss: 0.9846 - val_d_loss: 0.9984 - val_v_ccc: 0.0066 - val_a_ccc: 0.0154 - val_d_ccc: 0.0016
Epoch 2/50
6400/6400 [==============================] - 245s 38ms/step - loss: 1.6591 - v_loss: 0.6313 - a_loss: 0.4751 - d_loss: 0.5528 - v_ccc: 0.3687 - a_ccc: 0.5249 - d_ccc: 0.4472 - val_loss: 1.7558 - val_v_loss: 0.7157 - val_a_loss: 0.4825 - val_d_loss: 0.5575 - val_v_ccc: 0.2843 - val_a_ccc: 0.5175 - val_d_ccc: 0.4425
Epoch 3/50
6400/6400 [==============================] - 246s 38ms/step - loss: 1.1245 - v_loss: 0.3709 - a_loss: 0.3393 - d_loss: 0.4143 - v_ccc: 0.6291 - a_ccc: 0.6607 - d_ccc: 0.5857 - val_loss: 1.6459

6400/6400 [==============================] - 246s 38ms/step - loss: 0.3356 - v_loss: 0.0885 - a_loss: 0.1223 - d_loss: 0.1248 - v_ccc: 0.9115 - a_ccc: 0.8777 - d_ccc: 0.8752 - val_loss: 1.7114 - val_v_loss: 0.5808 - val_a_loss: 0.5012 - val_d_loss: 0.6294 - val_v_ccc: 0.4192 - val_a_ccc: 0.4988 - val_d_ccc: 0.3706
Epoch 25/50
2039/2039 [==============================] - 11s 5ms/step
[0.46843066811561584, 0.5685243606567383, 0.4313680827617645]
0
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 243s 38ms/step - loss: 3.0021 - v_loss: 1.0014 - a_loss: 1.0005 - d_loss: 1.0002 - v_ccc: -0.0014 - a_ccc: -4.8543e-04 - d_ccc: -1.7195e-04 - val_loss: 3.0001 - val_v_loss: 1.0000 - val_a_loss: 1.0000 - val_d_loss: 1.0001 - val_v_ccc: 1.6341e-05 - val_a_ccc: -2.1388e-07 - val_d_ccc: -7.8576e-05
Epoch 2/50
6400/6400 [==============================] - 238s 37ms/step - loss: 2.6178 - v_loss: 0.9223 - a_loss: 0.8334 - d_loss: 0.8621 - v_ccc: 0.07

Epoch 4/50
6400/6400 [==============================] - 248s 39ms/step - loss: 1.0372 - v_loss: 0.3148 - a_loss: 0.3284 - d_loss: 0.3940 - v_ccc: 0.6852 - a_ccc: 0.6716 - d_ccc: 0.6060 - val_loss: 1.6305 - val_v_loss: 0.6541 - val_a_loss: 0.4250 - val_d_loss: 0.5515 - val_v_ccc: 0.3459 - val_a_ccc: 0.5750 - val_d_ccc: 0.4485
Epoch 5/50
6400/6400 [==============================] - 245s 38ms/step - loss: 0.9470 - v_loss: 0.2648 - a_loss: 0.3137 - d_loss: 0.3686 - v_ccc: 0.7352 - a_ccc: 0.6863 - d_ccc: 0.6314 - val_loss: 1.6205 - val_v_loss: 0.6392 - val_a_loss: 0.4294 - val_d_loss: 0.5519 - val_v_ccc: 0.3608 - val_a_ccc: 0.5706 - val_d_ccc: 0.4481
Epoch 6/50
6400/6400 [==============================] - 241s 38ms/step - loss: 0.8689 - v_loss: 0.2417 - a_loss: 0.2954 - d_loss: 0.3318 - v_ccc: 0.7583 - a_ccc: 0.7046 - d_ccc: 0.6682 - val_loss: 1.5966 - val_v_loss: 0.6117 - val_a_loss: 0.4271 - val_d_loss: 0.5578 - val_v_ccc: 0.3883 - val_a_ccc: 0.5729 - val_d_ccc: 0.4422
Epoch 7/50
6400/640

6400/6400 [==============================] - 250s 39ms/step - loss: 0.6766 - v_loss: 0.1784 - a_loss: 0.2419 - d_loss: 0.2563 - v_ccc: 0.8216 - a_ccc: 0.7581 - d_ccc: 0.7437 - val_loss: 1.6313 - val_v_loss: 0.5864 - val_a_loss: 0.4591 - val_d_loss: 0.5858 - val_v_ccc: 0.4136 - val_a_ccc: 0.5409 - val_d_ccc: 0.4142
Epoch 11/50
6400/6400 [==============================] - 251s 39ms/step - loss: 0.6442 - v_loss: 0.1624 - a_loss: 0.2333 - d_loss: 0.2485 - v_ccc: 0.8376 - a_ccc: 0.7667 - d_ccc: 0.7515 - val_loss: 1.6511 - val_v_loss: 0.5844 - val_a_loss: 0.4726 - val_d_loss: 0.5941 - val_v_ccc: 0.4156 - val_a_ccc: 0.5274 - val_d_ccc: 0.4059
Epoch 12/50
6400/6400 [==============================] - 250s 39ms/step - loss: 0.6073 - v_loss: 0.1598 - a_loss: 0.2174 - d_loss: 0.2301 - v_ccc: 0.8402 - a_ccc: 0.7826 - d_ccc: 0.7699 - val_loss: 1.6323 - val_v_loss: 0.5881 - val_a_loss: 0.4618 - val_d_loss: 0.5824 - val_v_ccc: 0.4119 - val_a_ccc: 0.5382 - val_d_ccc: 0.4176
Epoch 13/50
6400/6400 [=====

Epoch 2/50
6400/6400 [==============================] - 246s 38ms/step - loss: 2.9202 - v_loss: 0.9858 - a_loss: 0.9618 - d_loss: 0.9725 - v_ccc: 0.0142 - a_ccc: 0.0382 - d_ccc: 0.0275 - val_loss: 2.1279 - val_v_loss: 0.9482 - val_a_loss: 0.5463 - val_d_loss: 0.6334 - val_v_ccc: 0.0518 - val_a_ccc: 0.4537 - val_d_ccc: 0.3666
Epoch 3/50
6400/6400 [==============================] - 245s 38ms/step - loss: 1.4406 - v_loss: 0.5503 - a_loss: 0.3881 - d_loss: 0.5023 - v_ccc: 0.4497 - a_ccc: 0.6119 - d_ccc: 0.4977 - val_loss: 1.8120 - val_v_loss: 0.6905 - val_a_loss: 0.5250 - val_d_loss: 0.5966 - val_v_ccc: 0.3095 - val_a_ccc: 0.4750 - val_d_ccc: 0.4034
Epoch 4/50
6400/6400 [==============================] - 245s 38ms/step - loss: 1.1088 - v_loss: 0.3463 - a_loss: 0.3377 - d_loss: 0.4248 - v_ccc: 0.6537 - a_ccc: 0.6623 - d_ccc: 0.5752 - val_loss: 1.6463 - val_v_loss: 0.6379 - val_a_loss: 0.4389 - val_d_loss: 0.5695 - val_v_ccc: 0.3621 - val_a_ccc: 0.5611 - val_d_ccc: 0.4305
Epoch 5/50
6400/640

...

6400/6400 [==============================] - 303s 47ms/step - loss: 1.1307 - v_loss: 0.3662 - a_loss: 0.3397 - d_loss: 0.4247 - v_ccc: 0.6338 - a_ccc: 0.6603 - d_ccc: 0.5753 - val_loss: 1.6487 - val_v_loss: 0.6756 - val_a_loss: 0.4543 - val_d_loss: 0.5189 - val_v_ccc: 0.3244 - val_a_ccc: 0.5457 - val_d_ccc: 0.4811
Epoch 4/50
6400/6400 [==============================] - 304s 47ms/step - loss: 1.0054 - v_loss: 0.2914 - a_loss: 0.3221 - d_loss: 0.3919 - v_ccc: 0.7086 - a_ccc: 0.6779 - d_ccc: 0.6081 - val_loss: 1.5836 - val_v_loss: 0.6162 - val_a_loss: 0.4344 - val_d_loss: 0.5329 - val_v_ccc: 0.3838 - val_a_ccc: 0.5656 - val_d_ccc: 0.4671
Epoch 5/50
6400/6400 [==============================] - 300s 47ms/step - loss: 0.9324 - v_loss: 0.2526 - a_loss: 0.3071 - d_loss: 0.3726 - v_ccc: 0.7474 - a_ccc: 0.6929 - d_ccc: 0.6274 - val_loss: 1.5940 - val_v_loss: 0.6098 - val_a_loss: 0.4274 - val_d_loss: 0.5568 - val_v_ccc: 0.3902 - val_a_ccc: 0.5726 - val_d_ccc: 0.4432
Epoch 6/50
6400/6400 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 299s 47ms/step - loss: 0.7244 - v_loss: 0.1892 - a_loss: 0.2595 - d_loss: 0.2757 - v_ccc: 0.8108 - a_ccc: 0.7405 - d_ccc: 0.7243 - val_loss: 1.6039 - val_v_loss: 0.5618 - val_a_loss: 0.4358 - val_d_loss: 0.6063 - val_v_ccc: 0.4382 - val_a_ccc: 0.5642 - val_d_ccc: 0.3937
Epoch 10/50
6400/6400 [==============================] - 299s 47ms/step - loss: 0.7008 - v_loss: 0.1807 - a_loss: 0.2516 - d_loss: 0.2684 - v_ccc: 0.8193 - a_ccc: 0.7484 - d_ccc: 0.7316 - val_loss: 1.5741 - val_v_loss: 0.5714 - val_a_loss: 0.4360 - val_d_loss: 0.5666 - val_v_ccc: 0.4286 - val_a_ccc: 0.5640 - val_d_ccc: 0.4334
Epoch 11/50
6400/6400 [==============================] - 301s 47ms/step - loss: 0.6552 - v_loss: 0.1680 - a_loss: 0.2432 - d_loss: 0.2439 - v_ccc: 0.8320 - a_ccc: 0.7568 - d_ccc: 0.7561 - val_loss: 1.6471 - val_v_loss: 0.5771 - val_a_loss: 0.4535 - val_d_loss: 0.6165 - val_v_ccc: 0.4229 - val_a_ccc: 0.5465 - val_d_ccc: 0.3835
Epoch 12/50
6400/6400 [=====

6400/6400 [==============================] - 288s 45ms/step - loss: 0.4517 - v_loss: 0.1242 - a_loss: 0.1598 - d_loss: 0.1676 - v_ccc: 0.8758 - a_ccc: 0.8402 - d_ccc: 0.8324 - val_loss: 1.6530 - val_v_loss: 0.5545 - val_a_loss: 0.4739 - val_d_loss: 0.6246 - val_v_ccc: 0.4455 - val_a_ccc: 0.5261 - val_d_ccc: 0.3754
Epoch 19/50
2039/2039 [==============================] - 13s 6ms/step
[0.3666301667690277, 0.5800116062164307, 0.42817217111587524]
26
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 300s 47ms/step - loss: 2.9808 - v_loss: 0.9940 - a_loss: 0.9952 - d_loss: 0.9917 - v_ccc: 0.0060 - a_ccc: 0.0048 - d_ccc: 0.0083 - val_loss: 2.4915 - val_v_loss: 0.9396 - val_a_loss: 0.7437 - val_d_loss: 0.8083 - val_v_ccc: 0.0604 - val_a_ccc: 0.2563 - val_d_ccc: 0.1917
Epoch 2/50
6400/6400 [==============================] - 286s 45ms/step - loss: 1.5524 - v_loss: 0.6224 - a_loss: 0.4221 - d_loss: 0.5079 - v_ccc: 0.3776 - a_ccc: 0.5779 - d_c

Epoch 9/50
6400/6400 [==============================] - 293s 46ms/step - loss: 0.7100 - v_loss: 0.1830 - a_loss: 0.2554 - d_loss: 0.2717 - v_ccc: 0.8170 - a_ccc: 0.7446 - d_ccc: 0.7283 - val_loss: 1.6239 - val_v_loss: 0.6001 - val_a_loss: 0.4396 - val_d_loss: 0.5841 - val_v_ccc: 0.3999 - val_a_ccc: 0.5604 - val_d_ccc: 0.4159
Epoch 10/50
6400/6400 [==============================] - 296s 46ms/step - loss: 0.6691 - v_loss: 0.1710 - a_loss: 0.2457 - d_loss: 0.2524 - v_ccc: 0.8290 - a_ccc: 0.7543 - d_ccc: 0.7476 - val_loss: 1.6067 - val_v_loss: 0.6171 - val_a_loss: 0.4396 - val_d_loss: 0.5500 - val_v_ccc: 0.3829 - val_a_ccc: 0.5604 - val_d_ccc: 0.4500
Epoch 11/50
6400/6400 [==============================] - 300s 47ms/step - loss: 0.6314 - v_loss: 0.1615 - a_loss: 0.2314 - d_loss: 0.2384 - v_ccc: 0.8385 - a_ccc: 0.7686 - d_ccc: 0.7616 - val_loss: 1.6548 - val_v_loss: 0.6125 - val_a_loss: 0.4444 - val_d_loss: 0.5980 - val_v_ccc: 0.3875 - val_a_ccc: 0.5556 - val_d_ccc: 0.4020
Epoch 12/50
6400/